In [1]:
# Check that imports for the rest of the file work.
!pip install tensorflow
!pip install tensorflow-gan
!pip install tensorflow-datasets
!pip install keras
!pip install opencv-python
import tensorflow.compat.v1 as tf
import tensorflow_gan as tfgan
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

# Allow matplotlib images to render immediately.
%matplotlib inline
tf.logging.set_verbosity(tf.logging.ERROR)  # Disable noisy outputs.
#evaluates operations immediately without building graphs

tf.disable_v2_behavior()
tf.compat.v1.enable_resource_variables()

In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# import os
# from PIL import Image, ImageOps
# import matplotlib.pyplot as plt
# import numpy as np
# import pathlib
# shirtDir = os.listdir("SixDollarImages/")

# import tensorflow_datasets as tfds
# import tensorflow.compat.v1 as tf
# import gc
# import cv2

# g = tf.Graph()

# # with tf.Session(graph=g) as session:
# def input_fn(mode, params):

#   shirtListTrain = []
#   shirtListTest = []
#   index = 0

#   for file in shirtDir:
#     im = Image.open("SixDollarImages/" + file)
#     im = im.resize((112, 112))
#     # im = im.resize((32, 32))
#     # im = im.resize((28, 28))
#     im = im.convert("RGB")

#     im = np.array(im).reshape((112, 112, 3))
#     # im = np.array(im).reshape((32, 32, 3))
#     # im = np.array(im).reshape((28, 28, 1))
#     # im = tf.convert_to_tensor(im, dtype=tf.uint8)
#     im = tf.convert_to_tensor(im, dtype=tf.float32)

#     if index < 20:
#       shirtListTrain.append(im)
#     else:
#       shirtListTest.append(im)
#     index = index + 1

#   datasetTrain = tf.data.Dataset.from_tensor_slices(shirtListTrain)
#   datasetTest = tf.data.Dataset.from_tensor_slices(shirtListTest)

#   assert 'batch_size' in params
#   assert 'noise_dims' in params
#   bs = params['batch_size']
#   nd = params['noise_dims']
#   split = 'train' if mode == tf.estimator.ModeKeys.TRAIN else 'test'
#   shuffle = (mode == tf.estimator.ModeKeys.TRAIN)
#   just_noise = (mode == tf.estimator.ModeKeys.PREDICT)
  
#   #for each tensor call tf.random_normal
#   #lambda is an unnamed python function 
#   # _ is used to ignore certian values, here we are ignoring the index
#   noise_ds = (tf.data.Dataset.from_tensors(0).repeat()
#               .map(lambda _: tf.random_normal([bs, nd])))
  
#   if just_noise:
#     return noise_ds
  
#   #define everything as tensors
#   def process_path(tensorInput):
#     #{'image': <tf.Tensor 'args_0:0' shape=(28, 28, 1) dtype=uint8>, 'label': <tf.Tensor 'args_1:0' shape=() dtype=int64>}
#     image = (tf.cast(tensorInput, tf.float32) - 127.5) / 127.5
#     #Tensor("truediv:0", shape=(28, 28, 1), dtype=float32)

#     return image

#   if split == 'train':
#     #returns a map() object (which is an iterator)
#     #stores data in a temporary location instead of retrieving from source everytime
    
#     images_ds = datasetTrain.map(process_path)
      
#   else:
#     images_ds = datasetTest.map(process_path)

#   if shuffle:
#     images_ds = images_ds.shuffle(
#         buffer_size=5, reshuffle_each_iteration=True)
#   images_ds = (images_ds.batch(bs, drop_remainder=True)
#               .prefetch(tf.data.experimental.AUTOTUNE))


#   # <PrefetchDataset shapes: (5, 28, 28, 1), types: tf.float32>
#   return tf.data.Dataset.zip((noise_ds, images_ds))

In [4]:
# Art dataset

import os
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import tensorflow.compat.v1 as tf
import gc

trainFileString = "dataset/dataset_updated/training_set/drawings/"
testFileString = "dataset/dataset_updated/validation_set/drawings/"

train_shirtDir = os.listdir(trainFileString)
test_shirtDir = os.listdir(testFileString)

def input_fn(mode, params):

  shirtListTrain = []
  shirtListTest = []
  index = 0

  for file in train_shirtDir:
    im = Image.open(trainFileString + file)
    im = im.resize((224, 224))
    # im = im.resize((32, 32))
    # im = im.resize((28, 28))
    im = im.convert("RGB")

    im = np.array(im).reshape((224, 224, 3))
    # im = np.array(im).reshape((32, 32, 3))
    # im = np.array(im).reshape((28, 28, 1))
    # im = tf.convert_to_tensor(im, dtype=tf.uint8)
    im = tf.convert_to_tensor(im, dtype=tf.float32)

    shirtListTrain.append(im)


  for file in test_shirtDir:
    im = Image.open(testFileString + file)
    im = im.resize((224, 224))
    # im = im.resize((32, 32))
    # im = im.resize((28, 28))
    im = im.convert("RGB")

    im = np.array(im).reshape((224, 224, 3))
    # im = np.array(im).reshape((32, 32, 3))
    # im = np.array(im).reshape((28, 28, 1))
    # im = tf.convert_to_tensor(im, dtype=tf.uint8)
    im = tf.convert_to_tensor(im, dtype=tf.float32)

    shirtListTest.append(im)


  datasetTrain = tf.data.Dataset.from_tensor_slices(shirtListTrain)
  datasetTest = tf.data.Dataset.from_tensor_slices(shirtListTest)

  assert 'batch_size' in params
  assert 'noise_dims' in params
  bs = params['batch_size']
  nd = params['noise_dims']
  split = 'train' if mode == tf.estimator.ModeKeys.TRAIN else 'test'
  shuffle = (mode == tf.estimator.ModeKeys.TRAIN)
  just_noise = (mode == tf.estimator.ModeKeys.PREDICT)
  
  #for each tensor call tf.random_normal
  #lambda is an unnamed python function 
  # _ is used to ignore certian values, here we are ignoring the index
  noise_ds = (tf.data.Dataset.from_tensors(0).repeat()
              .map(lambda _: tf.random_normal([bs, nd])))
  
  if just_noise:
    return noise_ds
  
  def process_path(tensorInput):
    #{'image': <tf.Tensor 'args_0:0' shape=(28, 28, 1) dtype=uint8>, 'label': <tf.Tensor 'args_1:0' shape=() dtype=int64>}
    image = (tf.cast(tensorInput, tf.float32) - 127.5) / 127.5
    #Tensor("truediv:0", shape=(28, 28, 1), dtype=float32)

    return image

  if split == 'train':
    #returns a map() object (which is an iterator)
    #stores data in a temporary location instead of retrieving from source everytime
    
    images_ds = datasetTrain.map(process_path)
      
  else:
    images_ds = datasetTest.map(process_path)

  if shuffle:
    images_ds = images_ds.shuffle(
        buffer_size=5, reshuffle_each_iteration=True)
  images_ds = (images_ds.batch(bs, drop_remainder=True)
              .prefetch(tf.data.experimental.AUTOTUNE))


  return tf.data.Dataset.zip((noise_ds, images_ds))

In [5]:
# import tensorflow_datasets as tfds
# import tensorflow.compat.v1 as tf

# def input_fn(mode, params):
#   assert 'batch_size' in params
#   assert 'noise_dims' in params
#   bs = params['batch_size']
#   nd = params['noise_dims']
#   # split = 'train' if mode == tf.estimator.ModeKeys.TRAIN else 'test'
#   split = 'train' if mode == tf.estimator.ModeKeys.TRAIN else 'train'
#   shuffle = (mode == tf.estimator.ModeKeys.TRAIN)
#   just_noise = (mode == tf.estimator.ModeKeys.PREDICT)
  
#   noise_ds = (tf.data.Dataset.from_tensors(0).repeat()
#               .map(lambda _: tf.random.normal([bs, nd])))
  
#   if just_noise:
#     return noise_ds

#   def _preprocess(element):
#     # Map [0, 255] to [-1, 1].
#     images = (tf.cast(element['image'], tf.float32) - 127.5) / 127.5
#     return images
  
#   images_ds = (tfds.load('cifar10', split=split)
#                .map(_preprocess)
#                .cache()
#                .repeat())
#   if shuffle:
#     images_ds = images_ds.shuffle(
#         buffer_size=10000, reshuffle_each_iteration=True)
#   images_ds = (images_ds.batch(bs, drop_remainder=True)
#                .prefetch(tf.data.experimental.AUTOTUNE))

#   print(images_ds)
#   return tf.data.Dataset.zip((noise_ds, images_ds))

In [6]:
def _dense(inputs, units, l2_weight):
  return tf.layers.dense(
      inputs, units, None,
      kernel_initializer=tf.keras.initializers.glorot_uniform,
      kernel_regularizer=tf.keras.regularizers.l2(l=l2_weight),
      bias_regularizer=tf.keras.regularizers.l2(l=l2_weight))

def _batch_norm(inputs, is_training):
  return tf.layers.batch_normalization(
      inputs, momentum=0.999, epsilon=0.001, training=is_training)

def _deconv2d(inputs, filters, kernel_size, stride, l2_weight):
  return tf.layers.conv2d_transpose(
      inputs, filters, [kernel_size, kernel_size], strides=[stride, stride], 
      activation=tf.nn.relu, padding='same',
      kernel_initializer=tf.keras.initializers.glorot_uniform,
      kernel_regularizer=tf.keras.regularizers.l2(l=l2_weight),
      bias_regularizer=tf.keras.regularizers.l2(l=l2_weight))

def _conv2d(inputs, filters, kernel_size, stride, l2_weight):
  return tf.layers.conv2d(
      inputs, filters, [kernel_size, kernel_size], strides=[stride, stride], 
      activation=None, padding='same',
      kernel_initializer=tf.keras.initializers.glorot_uniform,
      kernel_regularizer=tf.keras.regularizers.l2(l=l2_weight),
      bias_regularizer=tf.keras.regularizers.l2(l=l2_weight))
    

In [7]:
def unconditional_generator(noise, mode, weight_decay=2.5e-5):
  # """Generator to produce unconditional MNIST images."""
  is_training = (mode == tf.estimator.ModeKeys.TRAIN)
  
  sf = 16

  net = _dense(noise, 4 * 4 * (sf * 8), weight_decay)
  net = _batch_norm(net, is_training)
  net = tf.reshape(net, [-1, 4, 4, (sf * 8)])

  net = _deconv2d(net, (sf * 4), 5, 2, weight_decay)
  net = tf.nn.relu(_batch_norm(net, is_training))

  net = _deconv2d(net, (sf * 2), 5, 2, weight_decay)
  net = tf.nn.relu(_batch_norm(net, is_training))

  net = _deconv2d(net, (sf * 2), 5, 2, weight_decay)
  net = tf.nn.relu(_batch_norm(net, is_training))

  net = _deconv2d(net, (sf * 2), 5, 2, weight_decay)
  net = tf.nn.relu(_batch_norm(net, is_training))

  net = _conv2d(net, 3, 5, 2, 0.0)
  net = tf.tanh(net)

  return net

In [8]:
_leaky_relu = lambda net: tf.nn.leaky_relu(net, alpha=0.01)

def unconditional_discriminator(img, unused_conditioning, mode, weight_decay=2.5e-5):
  is_training = (mode == tf.estimator.ModeKeys.TRAIN)
  
  sf = 16
  net = _conv2d(img, sf, 5, 2, weight_decay)
  net = _batch_norm(net, is_training)
  net = _conv2d(img, sf, 5, 2, weight_decay)
  net = _batch_norm(net, is_training)
  net = _leaky_relu(net)

  net = _conv2d(img, sf * 2, 5, 2, weight_decay)
  net = _batch_norm(net, is_training)
  net = _conv2d(img, sf * 2, 5, 2, weight_decay)
  net = _batch_norm(net, is_training)
  net = _leaky_relu(net)

  net = _conv2d(img, sf * 4, 5, 2, weight_decay)
  net = _batch_norm(net, is_training)
  net = _conv2d(img, sf * 4, 5, 2, weight_decay)
  net = _batch_norm(net, is_training)
  net = _leaky_relu(net)

  net = _conv2d(img, sf * 8, 5, 2, weight_decay)
  net = _batch_norm(net, is_training)
  net = _conv2d(img, sf * 8, 5, 2, weight_decay)
  net = _batch_norm(net, is_training)
  net = _leaky_relu(net)

  net = tf.keras.layers.GlobalMaxPooling2D()(net)
  net = _dense(net, 1, weight_decay)

  return net

In [9]:
from tensorflow_gan.examples.cifar import util

import tempfile
import tensorflow_hub as tfhub

def calculate_frechet_distance(real_images, generated_images, num_batches=1):
    frechet_distance = tfgan.eval.frechet_inception_distance(real_images, generated_images, num_batches=1)
    frechet_distance.shape.assert_is_compatible_with([])

    return frechet_distance

def calculate_inception_score(images, n_split=1, eps=1E-16):
    score = tfgan.eval.inception_score(images, num_batches=1)
    return score
    

In [10]:
from tensorflow_gan.examples.mnist import util as eval_util
import os

def get_eval_metric_ops_fn(gan_model):
  real_data_logits = tf.reduce_mean(gan_model.discriminator_real_outputs)
  gen_data_logits = tf.reduce_mean(gan_model.discriminator_gen_outputs)
  
  real_mnist_score = calculate_inception_score(gan_model.real_data)
  generated_mnist_score = calculate_inception_score(gan_model.generated_data)
  
  frechet_distance = calculate_frechet_distance(
      gan_model.real_data, gan_model.generated_data)
  return {
      'real_data_logits': tf.metrics.mean(real_data_logits),
      'gen_data_logits': tf.metrics.mean(gen_data_logits),
      'real_mnist_score': tf.metrics.mean(real_mnist_score),
      'mnist_score': tf.metrics.mean(generated_mnist_score),
      'frechet_distance': tf.metrics.mean(frechet_distance),
  }

In [11]:
train_batch_size = 5 #@param
noise_dimensions = 64 #@param
generator_lr = 0.001 #@param
discriminator_lr = 0.001 #@param

def gen_opt():
  gstep = tf.train.get_or_create_global_step()
  base_lr = generator_lr
  # Halve the learning rate at 1000 steps.
  lr = tf.cond(gstep < 1000, lambda: base_lr, lambda: base_lr / 2.0)
  return tf.train.AdamOptimizer(lr, 0.5)

gan_estimator = tfgan.estimator.GANEstimator(
    generator_fn=unconditional_generator,
    discriminator_fn=unconditional_discriminator,
    generator_loss_fn=tfgan.losses.wasserstein_generator_loss,
    discriminator_loss_fn=tfgan.losses.wasserstein_discriminator_loss,
    params={'batch_size': train_batch_size, 'noise_dims': noise_dimensions},
    generator_optimizer=gen_opt,
    discriminator_optimizer=tf.train.AdamOptimizer(discriminator_lr, 0.5),
    get_eval_metric_ops_fn=get_eval_metric_ops_fn)

In [12]:
# Disable noisy output.
tf.autograph.set_verbosity(0, False)

import time
from tensorflow.python.tools.inspect_checkpoint import print_tensors_in_checkpoint_file
steps_per_eval = 500 #@param
max_train_steps = 200000 #@param
batches_for_eval_metrics = 5 #@param|

# Used to track metrics.
steps = []
real_logits, fake_logits = [], []
real_mnist_scores, mnist_scores, frechet_distances = [], [], []

cur_step = 0
start_time = time.time()

while cur_step < max_train_steps:
  next_step = min(cur_step + steps_per_eval, max_train_steps)

  start = time.time()
  print("train")
  gan_estimator.train(input_fn, max_steps=next_step)
  steps_taken = next_step - cur_step
  time_taken = time.time() - start
  print('Time since start: %.2f min' % ((time.time() - start_time) / 60.0))
  print('Trained from step %i to %i in %.2f steps / sec' % (
      cur_step, next_step, steps_taken / time_taken))
  cur_step = next_step
    
  # Calculate some metrics.
  metrics = gan_estimator.evaluate(input_fn, steps=batches_for_eval_metrics)

  steps.append(cur_step)
  real_logits.append(metrics['real_data_logits'])
  fake_logits.append(metrics['gen_data_logits'])
  real_mnist_scores.append(metrics['real_mnist_score'])
  mnist_scores.append(metrics['mnist_score'])
  frechet_distances.append(metrics['frechet_distance'])
  print('Average discriminator output on Real: %.2f  Fake: %.2f' % (
      real_logits[-1], fake_logits[-1]))
  print('Inception Score: %.2f / %.2f  Frechet Distance: %.2f' % (
      mnist_scores[-1], real_mnist_scores[-1], frechet_distances[-1]))
  
  # Vizualize some images.
  print("predict")
  iterator = gan_estimator.predict(
      input_fn, hooks=[tf.train.StopAtStepHook(num_steps=21)])
  try:
    imgs = np.array([next(iterator) for _ in range(20)])
  except StopIteration:
    pass
  tiled = tfgan.eval.python_image_grid(imgs, grid_shape=(2, 10))
  plt.axis('off')
  plt.imshow((np.squeeze(tiled) * 255).astype(np.uint8))
  plt.show()
  
  
# Plot the metrics vs step.
plt.title('MNIST Frechet distance per step')
plt.plot(steps, frechet_distances)
plt.figure()
plt.title('MNIST Score per step')
plt.plot(steps, mnist_scores)
plt.plot(steps, real_mnist_scores)
plt.show()

train


C:\Users\Brad\anaconda3\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\core.py:171: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  warnings.warn('`tf.layers.dense` is deprecated and '
C:\Users\Brad\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
C:\Users\Brad\anaconda3\lib\site-packages\tensorflow\python\keras\legacy_tf_layers\normalization.py:307: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  warnings.warn(
C:\Users\Brad\an

MemoryError: 